In [5]:
import pandas as pd
data = pd.read_csv("diabetes.csv")
data.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [21]:
data.Pregnancies.max()

17

In [6]:
data.shape

(768, 9)

In [7]:
x = data.drop("Outcome",axis=1)
y = data["Outcome"]


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.33, random_state = 1)


In [9]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
rfc_predict = rfc.predict(x_test)

C:\Users\APSSDC\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix

In [11]:
rfc_cv_score = cross_val_score(rfc,x,y,cv = 10,scoring = 'roc_auc')
rfc_cv_score

array([0.7137037 , 0.81962963, 0.82333333, 0.65259259, 0.77518519,
       0.82888889, 0.81814815, 0.84740741, 0.78423077, 0.78884615])

In [12]:
print("== confusion matrix ==")
print(confusion_matrix(y_test,rfc_predict))
print("== classification report ==")
print(classification_report(y_test,rfc_predict))
print("== all AUC scrore == ")
print(rfc_cv_score)
print("== Mean AUC score == ")
print("mean AUC score - random Forest : ", rfc_cv_score.mean())

== confusion matrix ==
[[147  15]
 [ 48  44]]
== classification report ==
              precision    recall  f1-score   support

           0       0.75      0.91      0.82       162
           1       0.75      0.48      0.58        92

   micro avg       0.75      0.75      0.75       254
   macro avg       0.75      0.69      0.70       254
weighted avg       0.75      0.75      0.74       254

== all AUC scrore == 
[0.7137037  0.81962963 0.82333333 0.65259259 0.77518519 0.82888889
 0.81814815 0.84740741 0.78423077 0.78884615]
== Mean AUC score == 
mean AUC score - random Forest :  0.7851965811965812


In [14]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000 , num=10)]
max_features = ["auto","sqrt"]
max_features
max_depth = [int(x) for x in np.linspace(100,500, num = 11)]
max_depth.append(None)

random_grid = {"n_estimators": n_estimators, "max_features" :max_features, "max_depth": max_depth }
rfc_random = RandomizedSearchCV(estimator = rfc , param_distributions= random_grid ,n_iter=100, verbose=2, cv=3,random_state=42,n_jobs=-1)
rfc_random.fit(x_train,y_train)
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.6min finished


{'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 300}


In [18]:
rfc = RandomForestClassifier(n_estimators = 400, max_depth = 300, max_features = "sqrt")
rfc.fit(x_train,y_train)
rfc_predict = rfc.predict(x_test)
rfc_cv_score = cross_val_score(rfc,x,y,cv=10,scoring = 'roc_auc')
print("== confution matrix ==")
print(confusion_matrix(y_test,rfc_predict))
print("== Classification Report == ")
print(classification_report(y_test,rfc_predict))
print("== all AUC score ==")
print(rfc_cv_score)
print("== Mean AUC Score ==")
print("Mean AUC score - random forest :", rfc_cv_score.mean())


== confution matrix ==
[[147  15]
 [ 34  58]]
== Classification Report == 
              precision    recall  f1-score   support

           0       0.81      0.91      0.86       162
           1       0.79      0.63      0.70        92

   micro avg       0.81      0.81      0.81       254
   macro avg       0.80      0.77      0.78       254
weighted avg       0.81      0.81      0.80       254

== all AUC score ==
[0.78       0.82814815 0.82555556 0.73518519 0.81296296 0.85703704
 0.86777778 0.9062963  0.80769231 0.86384615]
== Mean AUC Score ==
Mean AUC score - random forest : 0.8284501424501423


In [25]:
y_pred = rfc.predict([[6,148,72,35,0,33.6,0.627,50]])
y_pred

array([1], dtype=int64)

In [26]:
 rfc.predict([[1,85,66,29,0,26.6,0.351,31]])

array([0], dtype=int64)

# Voting Classifiers

In [28]:
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [46]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
x,y = make_moons(n_samples=100, noise= 0.15, random_state=0)
x

array([[-3.22140161e-02,  9.91183435e-01],
       [ 1.89369937e+00, -1.43260983e-01],
       [ 9.37139867e-01,  2.60605533e-02],
       [-2.09992548e-03,  7.37796614e-02],
       [ 9.30798303e-01, -5.19403855e-01],
       [ 2.01988692e+00,  3.93408264e-01],
       [ 5.44042634e-01,  5.96292060e-01],
       [ 9.28075755e-01, -6.24226911e-01],
       [ 1.74199561e-03,  3.19609386e-01],
       [ 1.84154522e+00,  3.03519978e-01],
       [-1.30194649e-01,  4.61523379e-01],
       [ 1.95220698e+00, -1.21516672e-01],
       [ 1.36130855e+00, -6.10645836e-01],
       [ 9.48854620e-01,  9.69104083e-01],
       [ 6.84138531e-01,  4.90457831e-01],
       [-3.55948639e-01,  1.02909238e+00],
       [ 2.30422245e-01,  1.11016938e+00],
       [ 1.68179520e-01,  9.11612785e-01],
       [-3.77758742e-01,  8.16951549e-01],
       [ 1.01735851e+00,  5.21809968e-01],
       [-7.61970461e-01,  5.16251552e-01],
       [ 3.72609000e-01, -1.95418424e-01],
       [ 8.33035136e-01,  4.72017740e-01],
       [ 7.

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state = 1)

In [49]:
voting_clf = VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf)],voting = "hard")
voting_clf.fit(x_train,y_train)

C:\Users\APSSDC\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\APSSDC\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)), ('rf', RandomFo...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [50]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, voting_clf):
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.85
RandomForestClassifier 1.0
VotingClassifier 0.95


C:\Users\APSSDC\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\APSSDC\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\APSSDC\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Bagging Classicifier

In [61]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=10,bootstrap=True,n_jobs=-1)
bag_clf

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=10, n_estimators=500, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [62]:
bag_clf.fit(x_train,y_train)
y_pred = bag_clf.predict(x_test)
y_pred

array([0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
      dtype=int64)

In [63]:
bag_clf.oob_score_

AttributeError: 'BaggingClassifier' object has no attribute 'oob_score_'